In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# Lightweight text corpus
def load_lightweight_text():
    text = """
    hello world this is a test
    machine learning is fun
    deep learning with tensorflow
    natural language processing with python
    python is a versatile language
    deep learning is deeper than machine learning
    """
    return text.lower()

text = load_lightweight_text()

In [3]:
# Tokenize the text
vocab_size = 100  # Limit the vocabulary size
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [4]:
# Create input sequences
input_sequences = []
for line in text.split("\n"):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i + 1]
        input_sequences.append(n_gram_sequence)

In [5]:
# Pad sequences
max_seq_length = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_length, padding='pre')

In [6]:
# Split into predictors and label
predictors, labels = input_sequences[:, :-1], input_sequences[:, -1]
labels = to_categorical(labels, num_classes=total_words)

In [7]:
# Build the GRU model
model = Sequential([
    Embedding(total_words, 128, input_length=max_seq_length - 1),
    GRU(128, return_sequences=False),
    Dense(total_words, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

c:\Users\Parsa\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Train the model
epochs = 40
model.fit(predictors, labels, epochs=epochs, batch_size=16, verbose=2)

Epoch 1/40
2/2 - 2s - 1s/step - accuracy: 0.0400 - loss: 3.0457
Epoch 2/40
2/2 - 0s - 28ms/step - accuracy: 0.2400 - loss: 3.0124
Epoch 3/40
2/2 - 0s - 21ms/step - accuracy: 0.4000 - loss: 2.9844
Epoch 4/40
2/2 - 0s - 22ms/step - accuracy: 0.4400 - loss: 2.9515
Epoch 5/40
2/2 - 0s - 26ms/step - accuracy: 0.3600 - loss: 2.9152
Epoch 6/40
2/2 - 0s - 17ms/step - accuracy: 0.3200 - loss: 2.8742
Epoch 7/40
2/2 - 0s - 23ms/step - accuracy: 0.3200 - loss: 2.8257
Epoch 8/40
2/2 - 0s - 24ms/step - accuracy: 0.3200 - loss: 2.7605
Epoch 9/40
2/2 - 0s - 37ms/step - accuracy: 0.2400 - loss: 2.6950
Epoch 10/40
2/2 - 0s - 22ms/step - accuracy: 0.2000 - loss: 2.6083
Epoch 11/40
2/2 - 0s - 24ms/step - accuracy: 0.2000 - loss: 2.5350
Epoch 12/40
2/2 - 0s - 23ms/step - accuracy: 0.2000 - loss: 2.4939
Epoch 13/40
2/2 - 0s - 22ms/step - accuracy: 0.3200 - loss: 2.4608
Epoch 14/40
2/2 - 0s - 26ms/step - accuracy: 0.3200 - loss: 2.4123
Epoch 15/40
2/2 - 0s - 20ms/step - accuracy: 0.3200 - loss: 2.3481
Epoch 

In [9]:
# Function to generate text
def generate_text(seed_text, next_words, model, tokenizer, max_seq_length):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_length - 1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)

        for word, index in tokenizer.word_index.items():
            if index == predicted:
                seed_text += " " + word
                break

    return seed_text

In [14]:
# Test the model
seed_text = "Python"
generated_text = generate_text(seed_text, 10, model, tokenizer, max_seq_length)
print("Generated Text:", generated_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Generated Text: Python is a versatile language language processing with python python python
